In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [2]:
import sqlite3
import json
from datetime import datetime

In [3]:
#import tensorflow as tf

#from tensorflow import keras
#from tensorflow.keras import layers
#from tensorflow.keras.layers.experimental import preprocessing

#print(tf.__version__)

In [4]:
#from scipy.stats import spearmanr

## Read in data

read in station name

In [5]:
with open('../buoy_station_name.json','r') as outfile:
    stndata=json.load(outfile)

data=[]
for ids in stndata:
    info = stndata[ids]
    data.append([ids,info['STN'],info['LAT'],info['LON'],info['LOCATE'],info['OWNER']])
    
# set up column names
columns = ['ST', 'STN','LAT','LON','LOCATE','OWNER']

# list to DataFrame 
stdf = pd.DataFrame(data=data,columns=columns)
stdf

,ST,STN,LAT,LON,LOCATE,OWNER
0,4N11,將軍,null,null,臺南將軍安檢所樓頂,氣象局
1,46694A,龍洞浮標,25.097778,121.9225,東北角龍洞遊艇港外海約0.25公里，該處水深約27公尺。,中央氣象局
2,46699A,花蓮浮標,24.031944,121.631389,花蓮七星潭外海約1公里，該處水深約30公尺。,中央氣象局
3,46708A,龜山島浮標,24.846667,121.926111,宜蘭龜山島龜尾端西方1公里，該處水深約25公尺。,交通部觀光局
4,46714D,小琉球浮標,22.313611,120.362778,小琉球海子口西南方外海約1.2公里，該處水深約99公尺。,中央氣象局
...,...,...,...,...,...,...
97,1306,臺北港,25.156667,121.371667,臺北港第3港埠,氣象局
98,1821,福隆,25.021667,121.95,福隆漁港碼頭,氣象局
99,1315,彭佳嶼,25.625278,122.075556,彭佳嶼海水淡化廠附近,氣象局
100,1551,和平港,24.297778,121.753889,花蓮和平港防波堤,和平工業港公司


## read in data from local sqlite database
read in data 

In [6]:
def read_sql(filename):
    conn = sqlite3.connect(filename, timeout=10)
    df = pd.read_sql("SELECT * FROM status",con=conn)
    conn.close()
    return df

filename = '../buoy_data.sqlite.p8.db'
df = read_sql(filename)

In [7]:
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

df['H']        = df['H'].astype('float')/100.
df['T']        = df['T'].astype('float')
df['Tmean']    = df['Tmean'].astype('float')

df['Temp']     = df['Temp'].astype('float')
df['Zt']       = df['Zt'].astype('float')

df['Vm']       = df['Vm'].astype('float')
df['Dm']       = df['Dm'].astype('float')
df['Vg']       = df['Vg'].astype('float')
df['Vms']      = df['Vms'].astype('float')
df['Vgs']      = df['Vgs'].astype('float')
df['Zv']       = df['Zv'].astype('float')

df['P']        = df['P'].astype('float')

df['Vmsat']    = df['Vmsat'].astype('float')

In [8]:
# add STN station name variable 
def label_stname (row):
    if row['ST'] in stndata:
        return stndata[row['ST']]['STN']
    
df['STN'] = df.apply (lambda row: label_stname(row), axis=1)

In [9]:
df['STN']
df.to_json('../ML_data.json')

In [10]:
pd.read_json('../ML_data.json')

,ST,DATETIME,H,Hs,T,Tmean,D,P,Temp,Zt,...,Dm,Vg,Vms,Vgs,Zv,QC,Hsat,Tsat,Vmsat,STN
0,46694A,2010-01-01 00:00:00,1.50,S,87.0,63.0,56.0,1021.9,18.1,0.0,...,65.0,6.6,3.0,4.0,3.0,Q,NaN,NaN,NaN,龍洞浮標
1,46694A,2010-01-01 01:00:00,1.83,M,95.0,64.0,90.0,1021.8,18.1,0.0,...,69.0,6.4,3.0,4.0,3.0,Q,NaN,NaN,NaN,龍洞浮標
2,46694A,2010-01-01 02:00:00,2.07,M,80.0,64.0,67.0,1021.0,18.1,0.0,...,78.0,7.0,3.0,4.0,3.0,Q,NaN,NaN,NaN,龍洞浮標
3,46694A,2010-01-01 03:00:00,2.13,M,95.0,66.0,67.0,1020.5,18.0,0.0,...,64.0,7.5,3.0,4.0,3.0,Q,NaN,NaN,NaN,龍洞浮標
4,46694A,2010-01-01 04:00:00,2.32,M,87.0,67.0,56.0,1019.9,18.2,0.0,...,67.0,7.2,3.0,4.0,3.0,Q,NaN,NaN,NaN,龍洞浮標
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1896809,C0U75,2016-01-01 02:00:00,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,7.847967,龜山島
1896810,1306,2016-01-01 02:00:00,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,10.722089,臺北港
1896811,1821,2016-01-01 02:00:00,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,9.172004,福隆
1896812,1315,2016-01-01 02:00:00,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,11.670791,彭佳嶼
